# ML Pipeline Preparation

Experimental notebook to create ML pipeline.

### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database.
- Define feature and target variables X and Y

#### Code:

In [1]:
# import libraries

# general
import pandas as pd
import numpy as np
import sqlalchemy
import re

# tokenization
import nltk
nltk.download(['punkt', 'wordnet', 'stopwords', 'words', 'averaged_perceptron_tagger', 'maxent_ne_chunker'])
from nltk import sent_tokenize, pos_tag, ne_chunk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer

# Feature Engineering
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import GridSearchCV, train_test_split

# High dimensional output
from sklearn.datasets import make_multilabel_classification

from sklearn.multioutput import MultiOutputClassifier
from sklearn.multiclass import OneVsRestClassifier

# ML models
#from sklearn.neighbors import KNeighborsClassifier
#from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier
from sklearn.ensemble import RandomForestClassifier
#from sklearn.neighbors import KNeighborsClassifier, RadiusNeighborsClassifier
from sklearn.svm import LinearSVC

#metrics
from sklearn.metrics import classification_report

#save model
import pickle

import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!


In [2]:
def load_data(database_filepath):
    """
    Loads data from SQL Database and transforms it for model training
    
    :param:
        database_filepath: SQL database file (string)
    :returns:
        x: Features (dataframe)
        y: Targets (dataframe)
        category_names: Target labels (list)
    """
    
    # create engine for connection to  database
    engine = sqlalchemy.create_engine('sqlite:///{}'.format(database_filepath))
    
    # load data from database into pandas dataframe
    df = pd.read_sql_table('Categorized_Messages', con=engine)
    
    # create feature matrix (numpy array) containing only the messages
    X = df['message']
    # create target/label matrix containing all possible categories to predict
    Y = df.iloc[:, 4:]
    category_names = df.iloc[:, 4:].columns
    
    return X, Y, category_names

#### Test:

In [3]:
X, Y, category_names = load_data('Disaster_Response_Data.db')



In [4]:
# inspect X
X

0        Weather update - a cold front from Cuba that c...
1                  Is the Hurricane over or is it not over
2                          Looking for someone but no name
3        UN reports Leogane 80-90 destroyed. Only Hospi...
4        says: west side of Haiti, rest of the country ...
5                   Information about the National Palace-
6                           Storm at sacred heart of jesus
7        Please, we need tents and water. We are in Sil...
8          I would like to receive the messages, thank you
9        I am in Croix-des-Bouquets. We have health iss...
10       There's nothing to eat and water, we starving ...
11       I am in Petionville. I need more information r...
12       I am in Thomassin number 32, in the area named...
13       Let's do it together, need food in Delma 75, i...
14       More information on the 4636 number in order f...
15       A Comitee in Delmas 19, Rue ( street ) Janvier...
16       We need food and water in Klecin 12. We are dy.

In [5]:
# inspect Y
Y

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
7,1,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,1,1,0,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


### 2. Tokenization function to process text data

#### Code:

In [6]:
def tokenize(text):
    """
    Loads data from SQL Database and transforms it for model training
    
    :param database_filepath: SQL database file (string)
    
    :returns x: Features (dataframe)
    :returns y: Targets (dataframe)
    :returns category_names: Target labels (list)
    """
    
    # replace any urls in messages with 'urlplaceholder'
    #url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    
    #detected_urls = re.findall(url_regex, text)
    #for url in detected_urls:
    #    text = text.replace(url, 'urlplaceholder')
    
    # Normalize
    text = re.sub(r'[^a-zA-Z0-9]', ' ', text.lower())  # Punctuation Removal and Case Normalizing
    
    # Tokenize
    words = word_tokenize(text)
    
    # Stop Word Removal
    stop_words = stopwords.words('english')
    words = [w for w in words if w not in stop_words]
    
    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    lem = [lemmatizer.lemmatize(w) for w in words]
    lem = [lemmatizer.lemmatize(w, pos='v') for w in lem]
    
    return lem

#### Test:

In [7]:
# test result of tokenization
xx = tokenize(X[3])
print(X[3])
print(xx)

UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperately.
['un', 'report', 'leogane', '80', '90', 'destroy', 'hospital', 'st', 'croix', 'function', 'need', 'supply', 'desperately']


Alternative approach: Use Part-Of-Speech Tagging and Named Entities Recognition

In [8]:
class PartOfSpeechInterpreter(BaseEstimator, TransformerMixin):
    """
    Estimator Class to transform text data using Part of Speech Tagging.
    
    Args:
    None
    
    Returns:
    Class Object
    """
     
    def part_of_speech(self, text):    
        named_entities = ne_chunk(pos_tag(word_tokenize(text)))
    
        return named_entities
    
    def fit(self, X, Y=None):
        return self
    
    def transform(self, X):
        named_entities = pd.Series(X).apply(self.part_of_speech)
        
        return pd.DataFrame(named_entities)

In [9]:
# test result of POS tagging and named entity recognition
pos = PartOfSpeechInterpreter()
xx = pos.transform(X[3])
print(X[3])

xx.values

UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperately.


array([[ Tree('S', [('UN', 'NNP'), ('reports', 'NNS'), ('Leogane', 'NNP'), ('80-90', 'CD'), ('destroyed', 'NN'), ('.', '.'), ('Only', 'RB'), Tree('ORGANIZATION', [('Hospital', 'JJ'), ('St.', 'NNP'), ('Croix', 'NNP')]), ('functioning', 'NN'), ('.', '.'), ('Needs', 'NNP'), ('supplies', 'NNS'), ('desperately', 'RB'), ('.', '.')])]], dtype=object)

### 3. Build a machine learning pipeline
This machine pipeline takes in the `message` column as input and output classification results on the other 36 categories in the dataset. Predicting multiple target variables.

#### Code:

In [10]:
def build_model():
    """
    Builds a model using Random Forest Classifier. Data is transformed in pipeline using Tokenization, Count Vectorizer,
    Tfidf Transformer and 
    
    :return: cv: Trained model after performing grid search (GridSearchCV model) 
    """
    
    # define pipeline with estimators including a few transformers and a classifier
    pipeline = Pipeline([
        
        ('text_pipeline', Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize, ngram_range=(1, 2), max_df=0.5, max_features=None)),
            ('tfidf', TfidfTransformer()),
        ])),
        ('multi_clf', MultiOutputClassifier(OneVsRestClassifier(LinearSVC())))
    ])
    
    # define parameters to perform grid search on pipeline
    parameters = {
        #'text_pipeline__vect__ngram_range': ((1, 1), (1, 2)),
        'text_pipeline__vect__max_features': (None, 10000)
    }
    
    # create GridSearchCV object with pipeline
    cv = GridSearchCV(estimator=pipeline, param_grid=parameters, scoring='f1_macro', cv=2, n_jobs=-1, verbose=10)
    
    return cv

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

#### Code:

In [11]:
# split data into training set and test set
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)

In [12]:
# initiate model
model = build_model()

# fit model to training data
model.fit(X_train, Y_train)

Fitting 2 folds for each of 2 candidates, totalling 4 fits
[CV] text_pipeline__vect__max_features=None ..........................
[CV]  text_pipeline__vect__max_features=None, score=0.38399678111300106, total=  18.1s
[CV] text_pipeline__vect__max_features=None ..........................


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:   25.5s remaining:    0.0s


[CV]  text_pipeline__vect__max_features=None, score=0.3833914588208062, total=  18.1s
[CV] text_pipeline__vect__max_features=10000 .........................


[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:   51.1s remaining:    0.0s


[CV]  text_pipeline__vect__max_features=10000, score=0.3740392453548378, total=  16.6s
[CV] text_pipeline__vect__max_features=10000 .........................


[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:  1.3min remaining:    0.0s


[CV]  text_pipeline__vect__max_features=10000, score=0.36950969059664673, total=  16.5s


[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:  1.7min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:  1.7min finished


GridSearchCV(cv=2, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('text_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.5, max_features=None, min_df=1,
        ngram_range=(1, 2...ty='l2', random_state=None, tol=0.0001,
     verbose=0),
          n_jobs=1),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'text_pipeline__vect__max_features': (None, 10000)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='f1_macro', verbose=10)

In [13]:
# see parameters for tuning model
model.estimator.get_params().keys()

dict_keys(['memory', 'steps', 'text_pipeline', 'multi_clf', 'text_pipeline__memory', 'text_pipeline__steps', 'text_pipeline__vect', 'text_pipeline__tfidf', 'text_pipeline__vect__analyzer', 'text_pipeline__vect__binary', 'text_pipeline__vect__decode_error', 'text_pipeline__vect__dtype', 'text_pipeline__vect__encoding', 'text_pipeline__vect__input', 'text_pipeline__vect__lowercase', 'text_pipeline__vect__max_df', 'text_pipeline__vect__max_features', 'text_pipeline__vect__min_df', 'text_pipeline__vect__ngram_range', 'text_pipeline__vect__preprocessor', 'text_pipeline__vect__stop_words', 'text_pipeline__vect__strip_accents', 'text_pipeline__vect__token_pattern', 'text_pipeline__vect__tokenizer', 'text_pipeline__vect__vocabulary', 'text_pipeline__tfidf__norm', 'text_pipeline__tfidf__smooth_idf', 'text_pipeline__tfidf__sublinear_tf', 'text_pipeline__tfidf__use_idf', 'multi_clf__estimator__estimator__C', 'multi_clf__estimator__estimator__class_weight', 'multi_clf__estimator__estimator__dual',

### 5. Test model
Report the f1 score, precision and recall for each output category of the dataset. 

Accuracy here not a meaningful performance indicator, as there are 36 categories where model has to predict only one ore more to be true (=1). Therefore a model with bad performance would score high accuracy as most of the values will be zero in y_prediction and y_test.

In [14]:
def evaluate_model(model, X_test, Y_test, category_names):
    """
    Measures model's performance on test data and prints out results.
    
    :param model: trained model (GridSearchCV Object)
    :param X_test: Test features (dataframe)
    :param Y_test: Test targets (dataframe)
    :param category_names: Target labels (dataframe)
    """
    
    # predict target values Y_pred of test features
    Y_pred = model.predict(X_test)

    print(classification_report(Y_test, Y_pred, target_names=category_names))
    
    print("\nBest Parameters:", model.best_params_)
    
    
evaluate_model(model, X_test, Y_test, category_names)

                        precision    recall  f1-score   support

               related       0.86      0.94      0.90      4993
               request       0.74      0.64      0.69      1128
                 offer       1.00      0.04      0.08        23
           aid_related       0.70      0.75      0.73      2754
          medical_help       0.61      0.36      0.46       525
      medical_products       0.69      0.33      0.45       336
     search_and_rescue       0.69      0.19      0.30       185
              security       0.00      0.00      0.00       125
              military       0.65      0.37      0.48       233
           child_alone       0.00      0.00      0.00         0
                 water       0.74      0.70      0.72       427
                  food       0.81      0.75      0.78       711
               shelter       0.72      0.62      0.67       556
              clothing       0.83      0.41      0.55       116
                 money       0.49      

### 6. Improve your model
Use grid search to find better parameters. 

[CV]  features__text_pipeline__vect__max_df=0.5, features__text_pipeline__vect__max_features=None, features__text_pipeline__vect__ngram_range=(1, 2), score=0.40316628506226776, total=  19.0s
[CV] features__text_pipeline__vect__max_df=0.5, features__text_pipeline__vect__max_features=None, features__text_pipeline__vect__ngram_range=(1, 2) 


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

#### Results of parameter tuning:
- features__text_pipeline__vect__ngram_range': (1, 1)
- features__text_pipeline__vect__max_df': (0.5)
- features__text_pipeline__vect__max_features': 5000
- multi_clf__estimator__n_estimators': 100
- multi_clf__estimator__min_samples_split': 4

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [16]:
def save_model(model, model_filepath):
    """
    Function to save trained model as pickle file.
    
    :param model: Trained model (GridSearchCV Object)
    :param model_filepath: Filepath to store model (string)
    
    :return: None
    """
    # save model
    pickle.dump(model, open(model_filepath, 'wb'))
    
    
save_model(model, 'Trained_model')

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.